# Valuing Bermudan Swaptions with QL Example

In [95]:
# This program is distributed in the hope that it will be useful, but WITHOUT
# ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS
# FOR A PARTICULAR PURPOSE.  See the FinancePy license for more details.

We value a European swaption using Black's model and try to replicate a QL example at http://khandrikacm.blogspot.com/2014/03/european-style-interest-rate-swaption.html

In [96]:
import numpy as np
import matplotlib.pyplot as plt

In [97]:
from financepy.products.libor import *
from financepy.finutils import *
from financepy.market.curves.FinLiborCurve import FinLiborCurve
from financepy.market.curves.FinInterpolate import FinInterpMethods

## Building a Libor Discount Curve

In [98]:
valuationDate = FinDate(28, 2, 2014)
settlementDate = FinDate(4, 3, 2014)

We have a vector of dates and zero rates

In [99]:
depoDCCType = FinDayCountTypes.THIRTY_E_360_ISDA
depos = []

depo = FinLiborDeposit(settlementDate, "1W", 0.0023, depoDCCType); depos.append(depo)
depo = FinLiborDeposit(settlementDate, "1M", 0.0023, depoDCCType); depos.append(depo)
depo = FinLiborDeposit(settlementDate, "3M", 0.0023, depoDCCType); depos.append(depo)
depo = FinLiborDeposit(settlementDate, "6M", 0.0023, depoDCCType); depos.append(depo)

# No convexity correction provided so I omit interest rate futures

swaps = []
fixedDCCType = FinDayCountTypes.ACT_365_ISDA
fixedFreqType = FinFrequencyTypes.SEMI_ANNUAL

swap = FinLiborSwap(settlementDate, "3Y", 0.00790, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "4Y", 0.01200, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "5Y", 0.01570, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "6Y", 0.01865, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "7Y", 0.02160, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "8Y", 0.02350, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "9Y", 0.02540, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "10Y", 0.0273, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "15Y", 0.0297, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "20Y", 0.0316, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "25Y", 0.0335, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "30Y", 0.0354, fixedFreqType, fixedDCCType); swaps.append(swap)

liborCurve = FinLiborCurve("USD_LIBOR", valuationDate, depos, [], swaps, FinInterpMethods.FLAT_FORWARDS)

In [100]:
print(liborCurve)

TIME: DISCOUNT FACTOR
0.0: 1.0
0.030117018305671307: 0.9999552797777655
0.09582687642713597: 0.9998083700624047
0.26283943248585867: 0.9994253304349999
0.5147272219514732: 0.998851320980872
3.017177652077253: 0.9765490993012332
4.013777166919467: 0.952904546594766
5.010376681761681: 0.9238064180671846
6.0124520181140175: 0.8926595681653329
7.011789443711293: 0.8570211783146512
8.011126869308567: 0.8248728623117386
9.015940116415965: 0.7903114691475575
10.012539631258178: 0.7538665138498148
15.014702580754676: 0.6306063482407415
20.016865530251174: 0.5174165672827589
25.01081474748249: 0.4118731405338343
30.01297769697899: 0.31535955474364513



In [101]:
exerciseDate = settlementDate.addTenor("5Y")
swapMaturityDate = exerciseDate.addTenor("5Y")
swapFixedCoupon = 0.040852
swapFixedFrequencyType = FinFrequencyTypes.SEMI_ANNUAL
swapFixedDayCountType = FinDayCountTypes.THIRTY_E_360_ISDA
swapFloatFrequencyType = FinFrequencyTypes.QUARTERLY
swapFloatDayCountType = FinDayCountTypes.ACT_360
swapNotional = ONE_MILLION
swaptionType = FinLiborSwaptionType.PAYER  
calendarType = FinCalendarTypes.WEEKEND
busDayAdjustType = FinBusDayAdjustTypes.NONE
dateGenRuleType = FinDateGenRuleTypes.BACKWARD

In [102]:
swaption = FinLiborSwaption(exerciseDate,
                            swapMaturityDate,
                            swaptionType,
                            swapFixedCoupon,
                            swapFixedFrequencyType,
                            swapFixedDayCountType, 
                            swapNotional, 
                            swapFloatFrequencyType,
                            swapFloatDayCountType,
                            calendarType, 
                            busDayAdjustType,
                            dateGenRuleType)

In [103]:
print(swaption)

EXERCISE DATE: MON 4 MAR 2019
SWAPTION TYPE: FinLiborSwaptionType.PAYER
MATURITY DATE: MON 4 MAR 2024
SWAP NOTIONAL: 1000000
FIXED COUPON: 4.0852
FIXED FREQUENCY: FinFrequencyTypes.SEMI_ANNUAL
FIXED DAY COUNT: FinDayCountTypes.THIRTY_E_360_ISDA
FLOAT FREQUENCY: FinFrequencyTypes.SEMI_ANNUAL
FLOAT DAY COUNT: FinDayCountTypes.ACT_360



In [104]:
model = FinLiborModelBlack(0.1533)

In [105]:
swaption.value(settlementDate, liborCurve, model)

23135.37762019705

This is close to a value of 23134.40637208518 using QL

## Internals

In [106]:
print(swaption)

EXERCISE DATE: MON 4 MAR 2019
SWAPTION TYPE: FinLiborSwaptionType.PAYER
MATURITY DATE: MON 4 MAR 2024
SWAP NOTIONAL: 1000000
FIXED COUPON: 4.0852
FIXED FREQUENCY: FinFrequencyTypes.SEMI_ANNUAL
FIXED DAY COUNT: FinDayCountTypes.THIRTY_E_360_ISDA
FLOAT FREQUENCY: FinFrequencyTypes.SEMI_ANNUAL
FLOAT DAY COUNT: FinDayCountTypes.ACT_360
PV01: 4.160736741175373
FWD SWAP RATE: 4.0843705042814875
FWD DF TO EXPIRY: 0.9238064180671846


We can see that the forward swap rate almost equals the fixed coupon. The underlying swap is close to being ATM forward.

In [107]:
swaption.printFixedLeg()

START DATE: MON 4 MAR 2019
MATURITY DATE: MON 4 MAR 2024
COUPON (%): 4.0852
FIXED LEG FREQUENCY: FinFrequencyTypes.SEMI_ANNUAL
FIXED LEG DAY COUNT: FinDayCountTypes.THIRTY_E_360_ISDA
VALUATION DATE TUE 4 MAR 2014
PAYMENT_DATE     YEAR_FRAC        FLOW         DF         DF*FLOW       CUM_PV
 WED 4 SEP 2019  0.5000000     20426.00   0.90801437     18547.10     18547.10
 WED 4 MAR 2020  0.5000000     20426.00   0.89265957     18233.46     36780.57
 FRI 4 SEP 2020  0.5000000     20426.00   0.87451244     17862.79     54643.36
 THU 4 MAR 2021  0.5000000     20426.00   0.85702118     17505.51     72148.87
 SAT 4 SEP 2021  0.5000000     20426.00   0.84066128     17171.35     89320.22
 FRI 4 MAR 2022  0.5000000     20426.00   0.82487286     16848.85    106169.07
 SUN 4 SEP 2022  0.5000000     20426.00   0.80736018     16491.14    122660.21
 SAT 4 MAR 2023  0.5000000     20426.00   0.79049583     16146.67    138806.88
 MON 4 SEP 2023  0.5000000     20426.00   0.77187392     15766.30    154573.

Copyright (c) 2020 Dominic O'Kane